##### Carregando dataset

In [2]:
import pandas as pd
from tqdm import tqdm
from googletrans import Translator
from warnings import filterwarnings
filterwarnings('ignore')

df = pd.read_excel('data/stanfordMOOCForumPostsSet.xlsx')
df.head(3)

,Text,Opinion(1/0),Question(1/0),Answer(1/0),Sentiment(1-7),Confusion(1-7),Urgency(1-7),CourseType,forum_post_id,course_display_name,forum_uid,created_at,post_type,anonymous,anonymous_to_peers,up_count,comment_thread_id,reads
0,Interesting! How often we say those things to ...,1,0,0,6.5,2.0,1.5,Education,5225177f2c501f0a00000015,Education/EDUC115N/How_to_Learn_Math,30CADB93E6DE4711193D7BD05F2AE95C,2013-09-02 22:55:59,Comment,0.0,0.0,0.0,5221a8262cfae31200000001,41.0
1,"What is \Algebra as a Math Game\"" or are you j...",0,1,0,4.0,5.0,3.5,Education,5207d0e9935dfc0e0000005e,Education/EDUC115N/How_to_Learn_Math,37D8FAEE7D0B94B6CFC57D98FD3D0BA5,2013-08-11 17:59:05,Comment,0.0,0.0,0.0,520663839df35b0a00000043,55.0
2,I like the idea of my kids principal who says ...,1,0,0,5.5,3.0,2.5,Education,52052c82d01fec0a00000071,Education/EDUC115N/How_to_Learn_Math,CC11480215042B3EB6E5905EAB13B733,2013-08-09 17:53:06,Comment,0.0,0.0,0.0,51e59415e339d716000001a6,25.0


##### Pré-processamento

In [3]:
# removendo null
null_indices = df[df['Text'].isnull()].index.tolist()
df["Text"].iloc()[null_indices] = ""

# removendo int
int_indices = df[df["Text"].apply(lambda x: type(x) == int)].index.tolist()
df["Text"].iloc()[int_indices] = ""

# removendo float
float_indices = df[df["Text"].apply(lambda x: type(x) == float)].index.tolist()
df["Text"].iloc()[float_indices] = ""

# removendo caracteres especiais
def remove_non_ascii(text):
    return "".join(char for char in text if char.isascii())

df["Text"] = df["Text"].apply(lambda x: remove_non_ascii(x))

##### Traduzir com Transformer Helsinki

In [ ]:
from transformers import pipeline
# translator = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt", device=-1) # device=-1 para CPU

In [38]:
df["Text"][6]

'The Peer Review module is not fully set up yet.  You haven\'t done anything wrong -- Professor Boaler simply has a bit more work to do before it is fully ready for us to participate._x0007__x0007_You can read what the Tech Support team has to say about it <a href=\\https://class.stanford.edu/courses/Education/EDUC115N/How_to_Learn_Math/discussion/forum/i4x-edx-templates-course-Empty/threads/51e35e8cbf1<phoneRedac><zipRedac><zipRedac><zipRedac><zipRedac>2<zipRedac>\\">in this discussion</a>."'

In [102]:
# traduzir um documento
translator(df["Text"][6], max_length=2048)[0]["translation_text"]

'O módulo de revisão por pares ainda não está totalmente configurado. Você não fez nada de errado - o Professor Boaler simplesmente tem um pouco mais de trabalho a fazer antes de estar totalmente pronto para participarmos._x0007___x0007__Você pode ler o que a equipe de Suporte Técnico tem a dizer sobre isso a hrefhttps://class.stanford.edu/courses/EducationEDUC115N/How_Lear_Lear_'

In [103]:
# teste de 10 traduções
df["Text"][0:10].apply(lambda x: translator(x, max_length=2048)[0]["translation_text"])

0    Interessante! Quantas vezes dizemos essas cois...
1    O que é "Álgebra como um Jogo de Matemática" o...
2    Eu gosto da ideia do diretor dos meus filhos q...
3    A partir de suas respostas, parece que os alun...
4    Os meninos adoraram a matemática, porque há li...
5    A ideia generalizada de que ser bom em matemát...
6    O módulo de revisão por pares ainda não está t...
7    Gostaria de começar o ano de folga compartilha...
8    Pedir aos alunos que ilustrem como os erros po...
9    Eu acho que isso cai no professor. Alguém me d...
Name: Text, dtype: object

In [104]:
# melhor uso da GPU
translated_posts = translator(
    df["Text"][0:10]
    .apply(lambda x: x if type(x) in (str, list) else "no text")
    .to_list(),
    max_length=2048,
)
translated_posts = [post["translation_text"] for post in translated_posts]
translated_posts

['Interessante! Quantas vezes dizemos essas coisas aos outros sem realmente entender o que estamos dizendo. Deve ter sido uma experiência poderosa! Excelente!',
 'O que é "Álgebra como um Jogo de Matemática" ou você está apenas dizendo que cria jogos que incorporam álgebra?',
 'Eu gosto da ideia do diretor dos meus filhos que diz “Smart não significa fácil, inteligente significa trabalhar duro” e incorpora a ideia de cometer erros para trabalhar duro.',
 'A partir de suas respostas, parece que os alunos realmente gostaram do poder que sentiam ser "livres" para resolver a matemática da maneira que quisessem. Eles usaram linguagem acadêmica como números em decomposição e números amigáveis e foram capazes de explicar o significado. "',
 'Os meninos adoraram a matemática, porque há liberdade, você pode fazer qualquer coisa." Que ótima maneira de ver matemática e números! Não acho que os alunos percebam que a matemática pode ser ensinada de maneira diferente. Eles só pensaram que é isso que

In [ ]:
# traduzir em lotes
batch_size = 100
translated_posts = []

for i in tqdm(range(0, len(df), batch_size)):
    try:
        batch = df["Text"][i : i + batch_size]
        translated_batch = translator(
          batch.apply(lambda x: x if type(x) in (str, list) else "no text").to_list(),
          max_length=2048,
        )
        translated_batch = [post["translation_text"] for post in translated_batch]
        translated_posts.extend(translated_batch)
    except Exception as e:
        print(f"Error in Translating batch {i // batch_size + 1} of {len(df) // batch_size}: {e}")
        continue
    
df.insert(1, column="Text_PT", value=translated_posts)
df.to_excel('data/stanfordMOOCForumPostsSet_PT.xlsx', index=False)
df.head(3)

##### Traduzir com Google Translate

In [4]:
async def traduzir_texto(texto, idioma_destino='pt'):
  async with Translator() as translator:
    result = await translator.translate(texto, idioma_destino)
    return result.text

In [5]:
texto = "Hello, how are you?"
traducao = await traduzir_texto(texto)  # Traduzindo para português
traducao

'Olá, como vai?'

In [6]:
# traduzir em lotes
batch_size = 100
translated_posts = []

for i in tqdm(range(0, len(df), batch_size)):
    try:
        batch = df["Text"][i : i + batch_size]
        translated_batch = []
        for text in batch:
            translated_batch.append(await traduzir_texto(text))
        
        translated_posts.extend(translated_batch)
    except Exception as e:
        print(f"Error in Translating batch {i // batch_size + 1} of {len(df) // batch_size}: {e}")
        continue

df.insert(1, column="Text_PT", value=translated_posts)
df.to_excel("data/stanfordMOOCForumPostsSet_PT.xlsx", index=False)
df.head(3)

  0%|          | 0/297 [00:00<?, ?it/s]

100%|██████████| 297/297 [8:11:16<00:00, 99.25s/it]   


,Text,Text_PT,Opinion(1/0),Question(1/0),Answer(1/0),Sentiment(1-7),Confusion(1-7),Urgency(1-7),CourseType,forum_post_id,course_display_name,forum_uid,created_at,post_type,anonymous,anonymous_to_peers,up_count,comment_thread_id,reads
0,Interesting! How often we say those things to ...,Interessante! Quantas vezes dizemos essas cois...,1,0,0,6.5,2.0,1.5,Education,5225177f2c501f0a00000015,Education/EDUC115N/How_to_Learn_Math,30CADB93E6DE4711193D7BD05F2AE95C,2013-09-02 22:55:59,Comment,0.0,0.0,0.0,5221a8262cfae31200000001,41.0
1,"What is \Algebra as a Math Game\"" or are you j...",O que é \ Álgebra como um jogo de matemática \...,0,1,0,4.0,5.0,3.5,Education,5207d0e9935dfc0e0000005e,Education/EDUC115N/How_to_Learn_Math,37D8FAEE7D0B94B6CFC57D98FD3D0BA5,2013-08-11 17:59:05,Comment,0.0,0.0,0.0,520663839df35b0a00000043,55.0
2,I like the idea of my kids principal who says ...,Gosto da ideia do diretor dos meus filhos que ...,1,0,0,5.5,3.0,2.5,Education,52052c82d01fec0a00000071,Education/EDUC115N/How_to_Learn_Math,CC11480215042B3EB6E5905EAB13B733,2013-08-09 17:53:06,Comment,0.0,0.0,0.0,51e59415e339d716000001a6,25.0


In [8]:
df['Text'][0], df['Text_PT'][0]

('Interesting! How often we say those things to others without really understanding what we are saying. That must have been a powerful experience! Excellent!',
 'Interessante! Quantas vezes dizemos essas coisas para os outros sem realmente entender o que estamos dizendo. Essa deve ter sido uma experiência poderosa! Excelente!')

In [10]:
df["Text"][15000], df["Text_PT"][15000]


('I was talking about products with same/ different indices... Anyhow, smgross has explained it beautifully... Regards!',
 'Eu estava falando sobre produtos com os mesmos/ diferentes índices ... De qualquer forma, Smgross explicou lindamente ... Atenciosamente!')

In [9]:
df["Text"][29000], df["Text_PT"][29000]

('Thanks for sharing bmadhusu,_x0007__x0007_You are correct, in essence, you can define test outcome however you want. Test + can be a good thing, test - can be a bad thing, etc. The key is understanding how to set up a 2x2 and how to calculate the corresponding values (sensitivity, specificity, NPV, PPV, etc). _x0007__x0007_Josh',
 'Obrigado por compartilhar Bmadhusu, _x0007__x0007_ Você está correto, em essência, você pode definir o resultado do teste como quiser. O teste + pode ser uma coisa boa, teste - pode ser uma coisa ruim, etc. A chave é entender como configurar um 2x2 e como calcular os valores correspondentes (sensibilidade, especificidade, VPL, PPV, etc.). _x0007__X0007_Josh')